In [16]:
#using LinearAlgebra 
struct Quat{T}
    sc::T
    i::T
    j::T
    k::T
end 

function Quat( a, b, c, d )
    T = typeof( a )
    return Quat{T}( a, b, c, d )
end 

function Quat( a, b, c )
    T = typeof( a )
    return Quat{T}( 0, a, b, c )
end 


function string_term( a, var; first = false )
    if a == 0 
        return ""
    elseif a == 1 && var != ""
        return (first ? "" : "+") * var
    elseif a == 1 && var == ""
        return (first ? "" : "+")*"1"
    elseif a == -1 && var != ""
        return "-$(var)"
    elseif a == -1 && var == ""
        return "-1"
    else
        return (first || a < 0 ? "" : "+" ) * "$a$(var)"
    end
end

function Base.show(io::IO, m::MIME"text/html", q::Quat) 
    
    if q.sc == 0 && q.i == 0 && q.j == 0 && q.k == 0
        print( "0" )
    end 
        
    str = string_term( q.sc, ""; first = true )
    str *= string_term( q.i, "i"; first = length( str ) == 0 )
    str *= string_term( q.j, "j"; first = length( str ) == 0 )
    str *= string_term( q.k, "k"; first = length( str ) == 0 )
    
    print( str )
end

ii = Quat( 0.0, 1.0, 0.0, 0.0 )
jj = Quat( 0.0, 0.0, 1.0, 0.0 )
kk = Quat( 0.0, 0.0, 0.0, 1.0 )

Quat{Float64}(0.0, 0.0, 0.0, 1.0)

k

In [110]:
function Base.:+( p::Quat, q::Quat )
    return Quat( p.sc+q.sc, p.i+q.i, p.j+q.j, p.k+q.k )
end 

function Base.:+( a::Number, q::Quat )
    return Quat( a+q.sc, q.i, q.j, q.k )
end 

function Base.:+( q::Quat, a::Number )
    return a+q
end 

function Base.:*( p::Quat, q::Quat )
    return Quat( p.sc*q.sc-p.i*q.i-p.j*q.j-p.k*q.k, 
                 p.sc*q.i+p.i*q.sc+p.j*q.k-p.k*q.j,
                 p.sc*q.j+p.j*q.sc+p.k*q.i-p.i*q.k,
                 p.sc*q.k+p.k*q.sc+p.i*q.j-p.j*q.i )      
end 

function Base.:-( a::Number, q::Quat )
    return Quat( a-q.sc, -q.i, -q.j, -q.k )
end 

function Base.:-( q::Quat, a::Number )
    return Quat( q.sc-a, q.i, q.j, q.k )
end 


function Base.:*( a::Number, q::Quat )
    return Quat( a*(q.sc), a*(q.i), a*(q.j), a*(q.k) )
end 

function Base.:-( q::Quat )
    return -1*q
end

function Base.:-( p::Quat, q::Quat )
    return p+(-1)*q
end

function Base.:/( p::Quat, a::Number )
    return (1/a)*p
end

function conjugate( q::Quat )
    return Quat( q.sc, -q.i, -q.j, -q.k )
end 

function norm( q::Quat )
    return (q.sc^2+q.i^2+q.j^2+q.k^2)^(1/2)
end

function Base.inv( q::Quat )
    return conjugate( q )/(norm( q )^2)
end 

function Base.:^( p::Quat, n::Integer )
    
    if n == 0
        return Quat( 1,0,0,0 )
    elseif n == 1
        return p
    elseif n > 1
        return prod( p for i in 1:n )
    elseif n == -1
        return inv( p )
    elseif n < -1
        pinv = inv( p )
        return prod( pinv for i in 1:-n )
    else 
        throw( "invalid exponent" )
    end 
end


function Base.:/( p::Quat, q::Quat )
    return p*q^-1
end

function dotprod( u::Quat, v::Quat )
    return u.sc*v.sc+u.i*v.i+u.j*v.j+u.k*v.k
end

function crossprod( u::Quat, v::Quat )
    return Quat( 0.0, u.j*v.k-u.k*v.j, -u.i*v.k+u.k*v.i, u.i*v.j-u.j*v.i )
end

function angle( u::Quat, v::Quat )
    
    alpha = acos( dotprod( u, v ))
    if !isapprox( sin( alpha ), norm( crossprod( u, v ))) 
        alpha *= -1
    end 
    return alpha
end 

random_quat() = Quat( [ 2*(rand()-.5) for i in 1:4 ]... )
random_pure_quat() = Quat( [ 2*(rand()-.5) for i in 1:3 ]... )

random_pure_quat (generic function with 1 method)

In [3]:
quat( theta, u0 ) = cos(theta)+sin(theta)u0

quat (generic function with 1 method)

In [114]:
u0 =  random_pure_quat( )
u0 = u0/norm(u0)
theta = 2*pi*rand()
u = quat( theta, u0 )

v0 =  random_pure_quat( )
v0 = v0/norm(v0)
beta = 2*pi*rand()
v = quat( beta, v0 )


Quat{Float64}(0.8772364604283802, 0.20078878097532912, -0.2133123274535374, -0.38031290917596156)

0.8772364604283802+0.20078878097532912i-0.2133123274535374j-0.38031290917596156k

In [115]:
norm(u)

1.0

In [116]:
dotprod(u,v)

0.3887840414484012

In [117]:
w=u*v
cosa = w.sc
w0 = w-w.sc
sena = norm(w0)
w0 = w0/sena

Quat{Float64}(0.0, -0.6883851176815751, 0.4158837708611185, -0.5942782335647677)

-0.6883851176815751i+0.4158837708611185j-0.5942782335647677k

In [118]:
w0

Quat{Float64}(0.0, -0.6883851176815751, 0.4158837708611185, -0.5942782335647677)

-0.6883851176815751i+0.4158837708611185j-0.5942782335647677k

In [119]:
cosa-(cos(theta)*cos(beta)-sin(theta)*sin(beta)*dotprod(u0,v0))

1.1102230246251565e-16

In [120]:
cos(beta)

0.8772364604283802

In [121]:
u0*v0

Quat{Float64}(-0.4503991052973599, 0.7595766251579562, 0.4439261374225328, 0.15203151309546914)

-0.4503991052973599+0.7595766251579562i+0.4439261374225328j+0.15203151309546914k

In [129]:
function s(t,b,u0,v0) 
    phi = angle( u0, v0 )
    sin(t)^2*sin(b)^2*sin(phi)^2+sin(t+b)^2+(cos(t-b)^2-cos(t+b)^2)*(dotprod(u0,v0)-1)/2
    #sin(t)^2*sin(b)^2*sin(phi)^2+cos(t)^2*sin(b)^2+cos(b)^2*sin(t)^2+2*cos(t)*sin(t)*cos(b)*sin(b)*cos(phi)
end 


s (generic function with 1 method)

In [130]:
c(t,b,u0,v0) = (cos(t)*cos(b)-sin(t)*sin(b)*dotprod(u0,v0))

c (generic function with 1 method)

In [131]:
s(theta,beta,u0,v0)

0.4768950509175359

In [125]:
norm(w-w.sc)^2

0.47689505091753576

In [143]:
function a(t,b,u0,v0) 
    phi = angle(u0,v0)
    return cos(t)*cos(b)-sin(t)*sin(b)*cos(phi)
end 

function b(t,b,u0,v0)
    phi = angle(u0,v0)
    return (sin(t)^2*sin(b)^2*sin(phi)^2+sin(t+b)^2+(cos(t-b)^2-cos(t+b)^2)*(cos(phi)-1)/2)^(1/2)
end 

function ww( t, b, u0, v0 )
    
    return cos(t)*sin(b)*v0+cos(b)*sin(t)*u0+sin(t)*sin(b)*crossprod(u0,v0)
end

ww (generic function with 1 method)

In [144]:
w = u*v

Quat{Float64}(0.7232599457197008, -0.4753821608539977, 0.28719930250949843, -0.4103942162084855)

0.7232599457197008-0.4753821608539977i+0.28719930250949843j-0.4103942162084855k

In [145]:
a(theta,beta,u0,v0)

0.7232599457197006

In [146]:
ww(theta,beta,u0,v0)

Quat{Float64}(0.0, -0.47538216085399776, 0.2871993025094986, -0.4103942162084855)

-0.47538216085399776i+0.2871993025094986j-0.4103942162084855k

In [147]:
norm(w-w.sc)

0.6905758835331102

In [148]:
b(theta,beta,u0,v0)

0.6905758835331103